In [1]:
import torch
import torch.nn as nn

In [2]:
class ConstantVelocityModel(nn.Module):
    def __init__(self, frame_rate : int = 10):
        super(ConstantVelocityModel, self).__init__()
        self.FRAME_RATE = frame_rate

    def forward(self, x, n_past_frames = 30, n_future_frames = 20):
        # Calculate the mean velocity
        # x shape: [batch_size, n_frames, n_features]
        # Using the first two features which are X and Y positions
        velocities = (x[:, 1:n_past_frames, :2] - x[:, :n_past_frames-1, :2]) * self.FRAME_RATE  # Multiply by frame rate to get velocity
        mean_velocity = torch.mean(velocities, dim=1, keepdim=True)

        # Predict future positions
        initial_positions = x[:, n_past_frames-1, :2].unsqueeze(1)
        time_steps = torch.arange(1, n_future_frames+1, device=x.device).view(1, -1, 1)
        predictions = initial_positions + mean_velocity * time_steps / self.FRAME_RATE  # Divide by frame rate

        return predictions
    
model = ConstantVelocityModel()
model.to('cpu')

dummy_input = torch.randn(1, 30, 2)

# Export the wrapped model to ONNX format
torch.onnx.export(
    model,                   # Wrapped model to export
    dummy_input,                     # Model input
    "./model/model_cvm.onnx",              # Output file name
    export_params=True,              # Store the trained parameter weights inside the model file
    opset_version=13,                # Set the ONNX opset version (adjust as needed)
    do_constant_folding=True,        # Whether to execute constant folding for optimization
    input_names=['input'],           # The model's input names
    output_names=['output'],         # The model's output names
    dynamic_axes={
        'input': {0: 'batch_size'},  # Enable dynamic axes for input
        'output': {0: 'batch_size'}  # Enable dynamic axes for output
    }
)


In [11]:
model(dummy_input).shape

torch.Size([1, 40, 2])